https://spark.apache.org/docs/2.3.1/api/python/pyspark.html#module-pyspark

## Instalacja Sparka na platformie Colab

In [2]:
#Instalacja Sparka na platformie Colab
!git clone https://github.com/djkormo/colab-examples.git

Cloning into 'colab-examples'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 103 (delta 30), reused 96 (delta 23), pack-reused 0
Receiving objects: 100% (103/103), 6.84 MiB | 16.84 MiB/s, done.
Resolving deltas: 100% (30/30), done.


In [3]:
!bash colab-examples/spark/install.bash

mkdir: cannot create directory ‘content’: File exists
Reading package lists... Done
Building dependency tree       
Reading state information... Done
openjdk-8-jdk-headless is already the newest version (8u222-b10-1ubuntu1~18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
--2019-10-14 20:40:42--  http://apache.crihan.fr/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
Resolving apache.crihan.fr (apache.crihan.fr)... 195.221.21.36, 2001:660:7401:211::36
Connecting to apache.crihan.fr (apache.crihan.fr)|195.221.21.36|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 230091034 (219M) [application/x-gzip]
Saving to: ‘spark-2.4.4-bin-hadoop2.7.tgz’

spark-2.4.4-bin-had 100%[===================>] 219.43M  26.9MB/s    in 8.9s    

2019-10-14 20:40:51 (24.6 MB/s) - ‘spark-2.4.4-bin-hadoop2.7.tgz’ saved [230091034/230091034]



In [0]:
# based on https://www.tutorialspoint.com/pyspark/pyspark_sparkcontext.htm

import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_ME'] = '/content/spark-2.4.4-bin-hadoop2.7'

import findspark
findspark.init("spark-2.4.4-bin-hadoop2.7")

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("First App") \
    .master("local[*]")\
    .getOrCreate()

In [0]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

## SparkContext
Obiekt SparkContext udostępnia możliwość dostępu do zasobów klastra Spark. SparkContext reprezentuje połączenie do klastra Spark i może być używany do tworzenia obiektów RDD i dystrybucji tych obiektów w tym klastrze. Obiekt SparkContext zawiera długą listę metod, ale najczęściej będziemy używać tylko tych, które służą do tworzenia zbiorów RDD (Resilient Distributed Datasets — stałe rozproszone zbiory danych).

In [6]:
type(sc)

pyspark.context.SparkContext

In [7]:
spark.sparkContext

<SparkContext master=local[*] appName=First App>

In [0]:
#dir(sc)

## SparkSession
SparkSession jest punktem wejścia do funkcjonalności SparkSQL. Jest to jeden z pierwszych obiektów tworzonych podczas tworzenia aplikacji SparkSQL. SparkSession tworzy się używając metody SparkSession.builder (która zapewnia dostęp do Builder API, którego używa się konfigurowania sesji). 

Po utworzeniu, SparkSession pozwala na tworzenie DataFrame (opartego na RDD), tworzenie zestawów danych, uzyskiwanie dostępu do usług SparkSQL, wykonywanie zapytania SQL, korzystanie z interfejsu DataFrameReader w celu załadowania zbioru danych wybranego formatu.

Obiekt SparkSession pod nazwą `spark` jest już utworzony po wystartowaniu kontenerów.

https://jaceklaskowski.gitbooks.io/mastering-spark-sql/spark-sql-SparkSession.html

In [10]:
type(spark)

pyspark.sql.session.SparkSession

## PySpark
PySpark jest oparty na Java API Sparka. Dane są przetwarzane w języku Python i buforowane/przetwarzane w JVM:

![PySpark-Data_Flow.png](attachment:PySpark-Data_Flow.png)

SparkContext używa biblioteki Py4J do uruchamiania maszyny JVM i tworzenia JavaSparkContext. Py4J jest używany tylko  do lokalnej komunikacji na nodach, pomiędzy obiektami Python i obiektami JavaSparkContext.

Transformacje na RDD w Pythonie są mapowane na transformacje obiektów PythonRDD w Javie poprzez serializację. Na zdalnych maszynach typu `Worker` obiekty PythonRDD uruchamiają podprocesy Pythona i komunikują się z nimi za pomocą potoków, wysyłając kod i dane do dalszego przetworzenia.

Tego rodzaju schemat wykonania i tłumaczenia obiektów pomiędzy Python a API Javy jest kosztowny, dlatego aplikacje napisane za pomocą PySpark są wolniejsze niż te napisane za pomocą języka Scala.

In [0]:
#sc = pyspark.SparkContext.getOrCreate(conf=conf)

# Resilient Distributed Datasets

RDD (Resilient Distributed Datasets — stałe rozproszone zbiory danych) to fundamentalne pojęcie stosowane w systemie Spark, oznaczające zbiór obiektów, które można rozdzielać między wiele instancji w klastrze. 
Zbiór RDD jest rozłożony pomiędzy instancje w klastrze jako zbiór **partycji**, z których każda zawiera pewien podzbiór danych. Partycja jest jednostką zrównoleglenia procesów w systemie Spark. Obiekty znajdujące się w partycji platforma przetwarza po kolei, natomiast same partycje przetwarzane są równolegle.

Zbiory RDD można tworzyć w systemie Spark na kilka sposobów:
1. Użycie metody parallelize obiektu SparkContext wraz z lokalnym zbiorem obiektów
2. Wykorzystując obiekt SparkContext do utworzenia zbioru RDD z wykorzystaniem zewnętrznego źródła danych, na przykład pliku w systemie HDFS, tabeli w bazie danych odczytanej za pomocą interfejsu JDBC lub lokalnego zbioru obiektów utworzonych w powłoce Spark.
3. Wykonując transformację jednego lub kilku istniejących zbiorów RDD, na przykład filtrowanie rekordów, agregację według wspólnego klucza lub łączenie kilku zbiorów RDD.

## 1. Użycie metody parallelize

In [11]:
data = [10, 20, 30, 40, 50]
RDD = sc.parallelize(data,4)
print('Typ obiektu RDD: {}'.format(type(RDD)))
print('Pierwsze cztery elementy obiektu RDD: {}'.format(RDD.take(4)))

Typ obiektu RDD: <class 'pyspark.rdd.RDD'>
Pierwsze cztery elementy obiektu RDD: [10, 20, 30, 40]


Pierwszym argumentem metody jest zbiór obiektów do zrównoleglenia. Drugi parametr oznacza liczbę partycji. Gdy obiekty w partycji mają być przetworzone, Spark pobiera podzbiór obiektów z kolekcji z procesu sterującego.

In [12]:
RDD.first()

10

Metoda first(), pobiera pierwszy element zbioru danych RDD na komputer klienta. Metoda first przydaje się do sprawdzania poprawności danych, jednak zazwyczaj będziemy chcieli pobierać większe porcje obiektów RDD do analizy na komputerze klienckim. Jeżeli będziemy pewni, że zbiór RDD zawiera niewielką liczbę rekordów, użyjemy metody collect, pobierającej na komputer klienta całą zawartość zbioru RDD w postaci tabeli.

## 2. Wykorzystanie obiektu SparkContext do utworzenia zbioru RDD z wykorzystaniem zewnętrznego źródła danych

In [15]:
# Uwaga na S3 w lokalizacji Frankfurt
# https://stackoverflow.com/questions/33828588/spark-cannot-read-files-stored-on-aws-s3-in-frankfurt-region-ireland-region-wor

csv_rdd = spark.sparkContext.textFile("s3a://sqlsaturday/export.csv")
type(csv_rdd)

pyspark.rdd.RDD

Jeżeli system Spark jest uruchomiony lokalnie, metoda textFile może odwoływać się do plików zapisanych w lokalnym systemie. Jeżeli w systemie Linux zostanie podana nazwa katalogu, a nie pojedynczego pliku, Spark przyjmie, że zbiór RDD opisują wszystkie pliki w tym katalogu (w systemie Windows zamiast nazwy pliku należy użyć symbolu \*). Należy zwrócić ponadto uwagę, że w tym momencie nie są jeszcze odczytywane ani ładowane do pamięci żadne dane, ani na lokalnym komputerze, ani w klastrze. Gdy nadejdzie moment przetworzenia obiektów zapisanych w partycji, Spark odczyta odpowiednią sekcję (zwaną również fragmentem) pliku wejściowego i wykona serię transformacji (np. count(), filter(), agg() itp.), zdefiniowanych za pomocą innych zbiorów RDD.

In [16]:
c = csv_rdd.count()

Py4JJavaError: ignored

In [0]:
print('Ilość elementów w zbiorze csv_rdd: {}'.format(c))

In [0]:
#http://kavita-ganesan.com/reading-csv-and-json-files-in-spark/
csv_rdd.take(5)

Kompromis pomiędzy wcześniej opisanymi metodami first() i collect() pozwala osiągnąć metoda take(), umożliwiająca pobranie na komputer klienta zadanej liczby rekordów.

## 3. Transformacje jednego RDD w  drugi zbiór RDD

Transformacje (ang. transformations) polegają na przekształceniach obiektów RDD (czyli operacje Map), filtrowaniu listy obiektów, albo grupowaniu. Transformacje są wykonywane dopiero wtedy, kiedy pojawi się akcja, która wymaga dostępu do przetworzonych danych (jest to tzw. leniwa ewaluacja, ang. *lazy evaluation*). Przykładową transformację może być zamiana typu int na float albo przefiltrowanie zbioru danych.

## Akcje
Akcje to komendy, których wynik jest obliczany w chwili ich uruchomienia (ang. eager evaluation). Takie uruchomienie obejmuje też wykonanie wszystkich transformacji, które poprzedzały akcję. Akcja składa się z jednego lub więcej zadań, które zostaną wykonane równolegle przez wiele workerów. Grafika poniżej prezentuje wybrane transformacje i akcje.

![trans_and_actions.png](attachment:trans_and_actions.png)

In [17]:
rdd = sc.parallelize([1,2,3,4,5,6,7,8,9,10])
rdd.filter(lambda x: x % 2 == 0) 

PythonRDD[12] at RDD at PythonRDD.scala:53

W tym momencie nie zauważysz efektu działania transformacji, stanie się to dopiero, gdy użyjesz akcji np. collect()

### Filter
filter pobiera predykat (czyli wyrażenie opisujące jakąś relację lub właściwość) i zwraca RDD z wszystkimi elementami pasującymi do predykatu.

In [20]:
# zastosowana akcja collect()
rdd = sc.parallelize([1,2,3,4,5,6,7,8,9,10])
rdd.collect()

rdd.filter(lambda x: x % 2 == 0).collect()

[2, 4, 6, 8, 10]

### Map
map transformuje elementy jednego RDD na drugi obiekt RDD

In [21]:
rdd = sc.parallelize([1,2,3,4,5,6,7,8,9,10])
rdd.map(lambda n: "Element %s" % str(n)).collect()

['Element 1',
 'Element 2',
 'Element 3',
 'Element 4',
 'Element 5',
 'Element 6',
 'Element 7',
 'Element 8',
 'Element 9',
 'Element 10']

In [22]:
rdd = sc.parallelize([1,2,3,4,5,6,7,8,9,10])
rdd.map(lambda n: n * 2).collect()

[2, 4, 6, 8, 10, 12, 14, 16, 18, 20]

### FlatMap
flatMap przekształca i spłaszcza RDD za pomocą funkcji

In [23]:
rdd = sc.parallelize([1,2,3])
rdd.flatMap(lambda n: [n, n * 2, n * 3]).collect()

[1, 2, 3, 2, 4, 6, 3, 6, 9]

### Distinct

Usuwa duplikaty z pierwotnego RDD

In [24]:
rdd = sc.parallelize(["alfa", "beta", "alfa", "alf"])
rdd.distinct().collect()

['alfa', 'alf', 'beta']

### Sample

Udostępnia próbkę danych z RDD. Ta metoda ma 2 argumenty:
- z powtórzeniem lub bez
- frakcja

In [25]:
rdd = sc.parallelize([1,2,3,4,5,6,7,8,9,10])
rdd.sample(False, 0.5).collect()

[4, 6, 7, 8, 9]

## Transformacje pomiędzy obiektami RDD

Transformacje tego typu pozwalają użytkownikowi na działanie na zestawach RDD. Parametrem wszystkich tych metod jest inny RDD.

### Union

Tworzy RDD zawierający elementy obydwu obiektów RDD

In [26]:
rdd_a = sc.parallelize([1,2,3])
rdd_b = sc.parallelize([3,4,5,6])
rdd_a.union(rdd_b).collect()

[1, 2, 3, 3, 4, 5, 6]

### Intersection

Tworzy RDD zawierający elementy występujące w obydwu zbiorach RDD

In [27]:
rdd_a = sc.parallelize([1,2,3])
rdd_b = sc.parallelize([3,4,5,6])
rdd_a.intersection(rdd_b).collect()

[3]

### Substract

Usuwa z RDD A elementy istniejące w zbiorze B 

In [28]:
rdd_a = sc.parallelize([1,2,3,4])
rdd_b = sc.parallelize([3,4,5,6])
rdd_a.subtract(rdd_b).collect()

[1, 2]

### Cartesian

Wynikiem jest iloczyn kartezjański zbiorów A i B 

In [29]:
rdd_a = sc.parallelize([1,2,3])
rdd_b = sc.parallelize([3,4,5])
rdd_a.cartesian(rdd_b).collect()

[(1, 3), (1, 4), (1, 5), (2, 3), (3, 3), (2, 4), (2, 5), (3, 4), (3, 5)]

## Akcje

Na tym etapie stosowana była jedna akcja collect()

### reduce()

Często używaną akcją jest redukcja, reduce() pobiera dwa elementy RDD i zwraca jeden tego samego typu.

In [30]:
rdd = sc.parallelize([1,2,3,4,1],2)
rdd.reduce(lambda a, b: a + b)

11

### fold()

fold jest podobny do akcji reduce(), ale dodatkowo należy podać wartość zerową. W matematyce ta wartość zerowa nazywana jest elementem tożsamości.

Agreguje elementy z każdej partycji, a następnie wyniki tych agregacji ze wszystkich partycji, używając danej funkcji asocjacyjnej i neutralnej "wartości zerowej". 
Uwaga! Zwróć uwagę, że wynik funkcji fold() będzie różnił się w zależności od partycjonowania (sposobu i ilości partycji) - tak się dzieje gdy wartość neutralna jest dobrana niepoprawnie. Poprawna wartość neutralna powinna wynosić:
- 0 gdy wykonywane jest dodawanie;
- 1 gdy wykonywane jest mnożenie;

In [31]:
rdd = sc.parallelize([1,2,3,4])
rdd.fold(0, lambda a, b: a + b)

10

In [32]:
rdd = sc.parallelize([1,2,3,4],2)
rdd.fold(1, lambda a, b: a * b)

24

In [33]:
# widok podziału RDD na partycje
rdd.glom().collect()

[[1, 2], [3, 4]]

In [34]:
rdd = sc.parallelize(['alice_', 'ada_', 'betty_', 'marta_'], 3)
rdd.fold('app+ ', lambda a, b: a + b)

'app+ app+ alice_app+ ada_app+ betty_marta_'

In [35]:
rdd.glom().collect()

[['alice_'], ['ada_'], ['betty_', 'marta_']]

### aggregate()

Akcje fold() i reduce() zwracają ten sam typ. Aggregate() pozwala pobrać RDD i wygenerować pojedynczą wartość, która jest innego typu niż to, co było przechowywane w oryginalnym RDD.

**aggregate(zeroValue, seqOp, combOp)**

Parametry:
1. zeroValue - Neutralna wartość inicjalizacji dla zakładanego wyniku w pożądanym formacie.
2. seqOp - (ang. Sequence operation) operacja, mająca być zastosowana do rekordów RDD. Uruchamia się jednokrotnie dla każdego rekordu w partycji.
3. combOp - (ang. Combiner operation) określa, w jaki sposób obiekty wynikowe (po jednym dla każdej partycji) zostają połączone.

Przykład: Utwórz RDD jako listę składającą się z czterech wartości typu int, oblicz sumę wartości elementów listy i jej długość. Zwróć wynik w postaci krotki (suma, długość)

In [0]:
rdd = sc.parallelize([1,2,3,4], 2)

zeroValue = (0, 0)
seqOp = (lambda local_result, list_element: (local_result[0] + list_element, local_result[1] + 1))
combOp = (lambda some_local_result, another_local_result: (some_local_result[0] + another_local_result[0], some_local_result[1] + another_local_result[1]) )

In [37]:
rdd.glom().collect()

[[1, 2], [3, 4]]

Pierwsza partycja ma podlistę [1, 2]. Zastosujemy seqOp do każdego elementu na tej liście, a to spowoduje wynik lokalny, parę (suma, długość), która będzie odzwierciedlać wynik lokalnie, tylko w tej pierwszej partycji.

Inicjalizacja `local_result` zostanie wykoanana za pomocą parametru `zeroValue`, który ustalno w wywołaniu akcji  aggregate() tj. (0, 0) i zmienna list_element jest pierwszym elementem listy, czyli jest to 1. W rezultacie otrzymamy:
```
0 + 1 = 1
0 + 1 = 1
```
Teraz zmienna `local_result` wynosi (1,1), czyli oznacza to, że dla pierwszej partycji po przetworzeniu pierwszego elementu, suma wynosi 1 i długość także 1. Oczywiście `local_result` w tym kroku zmienił swoją wartość z (0,0) na (1,1). W dalszym kroku, przetworzony zostanie drugi element pierwszej prartycji, czyli:
```
1 + 2 = 3
1 + 1 = 2
```
Zmienna `local_result` wynosi (3,2) i jest to końcowy wynik dla pierwszej partycji. Dla drugiej partycji przebieg jest taki sam, zaś zmienna `local_result` wynosi (7,2).

Kolejnym etapem jest uruchomie operacji sumatora (combOp) do każdego lokalnego wyniku, abyśmy mogli stworzyć ostateczny, globalny wynik, taki jak ten:
```
(3,2) + (7,2) = (10, 4)
```

In [38]:
rdd.aggregate(zeroValue=zeroValue, seqOp=seqOp, combOp=combOp)

(10, 4)

### first()

Zwraca pierwszy element obiektu RDD

In [39]:
rdd = sc.parallelize([4,3,2,1])
rdd.first()

4

### count()

Zwraca ilość elementów w RDD

In [40]:
rdd = sc.parallelize([1,2,3,4,5,6])
rdd.count()

6

In [41]:
rdd.glom().collect()

[[1, 2, 3], [4, 5, 6]]

### take(),  takeOrdered(), top()

- take() zwraca n elementów z  RDD; 
- takeOrdered() zwraca n elementów z RDD zgodnie z postawionym kryterium (domyślnie jest to naturalOrder); 
- top() zwraca n największych wartości z RDD;

In [42]:
rdd = sc.parallelize([1,2,3,4,5,6])
rdd.take(3)

[1, 2, 3]

In [43]:
# domyślne kryterium naturalOrder - pierwsze 3 elementy, z posortowanej listy elementów
rdd = sc.parallelize([4,2,1,5,6,3,7])
rdd.takeOrdered(3)

[1, 2, 3]

In [44]:
# kryterium 4 elementy z listy posortowanej malejąco
rdd = sc.parallelize([4,2,1,5,6,3,7])
rdd.takeOrdered(4, key = lambda x: -x)

[7, 6, 5, 4]

In [45]:
# pobierz 3 elementy o największych wartościach
rdd = sc.parallelize([4,2,1,5,6,3,7])
rdd.top(3)

[7, 6, 5]

### takeSample()

Zwraca n losowo wybranych elementów z RDD. Pierwszy argument `withReplacement` przybiera wartości True lub False. Gdy pierwszy argument jest True i gdy ilość elementów w RDD jest mniejsza niż n (drugi argument) to w wyniku wystąpią powtórzenia wartości. 

In [46]:
rdd = sc.parallelize([1,2,3,4,5,6])
rdd.takeSample(True, 10)

[1, 3, 5, 6, 1, 3, 2, 5, 3, 2]

In [47]:
rdd = sc.parallelize([1,2,3,4,5,6])
rdd.takeSample(False, 10)

[3, 6, 5, 4, 2, 1]

In [48]:
# za każdym razem wbrane będą dwa inne elementy
rdd = sc.parallelize([1,2,3,4,5,6])
rdd.takeSample(False, 2)

[4, 3]

In [49]:
# wybrane będą zawsze te same elementy
rdd = sc.parallelize([1,2,3,4,5,6])
rdd.takeSample(False, 2, seed=12)

[6, 2]

### countByValue()

Zwraca ilość wystąpień danego elementu w RDD

In [50]:
rdd = sc.parallelize([1,2,5,5,2,2,3,4,3,4])
rdd.countByValue()

defaultdict(int, {1: 1, 2: 3, 3: 2, 4: 2, 5: 2})

### foreach()

Stosuje określoną funkcję do każdego rekordu RDD metodą jeden-po-drugim. Uwaga: w przypadku notatnika Jupyter funkcja nie daje efektu, dlatego wypróbuj przykład w oknie terminala i wiersza poleceń PySpark.

In [0]:
def f(x): print('Element = {}'.format(x))

rdd = sc.parallelize([1, 2, 3, 4, 5])
rdd.foreach(f)

### mean(), stats()

`mean()` oblicza średnią elementów RDD; `stats()` zwraca obiekt StatCounter, który zwiera średnią, wariancję, i licznik obiektu RDD.

In [52]:
rdd = sc.parallelize([1, 2, 3, 4, 5])
rdd.mean()

3.0

In [53]:
rdd = sc.parallelize([1, 2, 3, 4, 5])
rdd.stats()

(count: 5, mean: 3.0, stdev: 1.4142135623730951, max: 5.0, min: 1.0)

## Spark DataFrame

In [54]:
df = spark.createDataFrame([('Alice', 13), ('Bob', 12)], ['name', 'age'])
type(df)

pyspark.sql.dataframe.DataFrame

In [55]:
print(df.collect())

[Row(name='Alice', age=13), Row(name='Bob', age=12)]


Akcja collect zwraca tabelę Array, zawierającą wszystkie obiekty ze zbioru RDD. Tabela ta jest umieszczona w pamięci drivera, a nie rozproszna w całym klastrze:

In [56]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)



In [0]:
from pyspark.sql.types import *

In [0]:
df = spark.read.load ("s3a://sqlsaturday/export.csv",
                                format='com.databricks.spark.csv',
                                header='true',
                                inferSchema='true')

In [59]:
df.show()

+-----+---+
| name|age|
+-----+---+
|Alice| 13|
|  Bob| 12|
+-----+---+



In [0]:
df_s3 = spark.read.format("CSV").option("inferSchema", "true").option("header", "true").option("delimiter", "\t").load("s3a://sqlsaturday/s3inputfile.dat")
#df_s3 = spark.read.csv("s3a://sqlsaturday/export.csv",  header=True)

In [0]:
df_s3.show()

In [0]:
df_s3.count()

In [0]:
df_s3.printSchema()

In [0]:
df_s3 = df_s3.drop('dt')

In [0]:
col_names = ['id', 'date', 'class', 't0', 'dt']
df_s3 = df_s3.toDF(*col_names)

In [0]:
display(df_s3.select("class"))

In [0]:
from pyspark.sql.functions import to_date, unix_timestamp, col

In [0]:
df_s3 = df_s3.withColumn("date_format", to_date(unix_timestamp(col("date"), "MM-dd-yy").cast("timestamp"))) #jeśli format daty jest źle określony po konwersji występuje wartość null
df_s3.printSchema()

In [0]:
#https://stackoverflow.com/questions/31674530/write-single-csv-file-using-spark-csv

In [0]:
df_s3.write.save("s3a://sqlsaturday/plik_2018-02-*.csv", format='csv', header=True,mode='overwrite')

In [0]:
df_s3.coalesce(1).write.mode("overwrite").format("csv").save("s3a://sqlsaturday/myfile.csv")

In [0]:
df_s3.repartition(1).write.mode("overwrite").format("csv").save("s3a://sqlsaturday/myfile_repartition.csv")

In [0]:
df_s3.write.format("com.databricks.spark.csv").mode("overwrite").save("s3a://sqlsaturday/myfile.csv")

In [0]:
df_s3.show()

In [0]:
# ----- Działa na platformie Databricks
# save_location = "s3a://sqlsaturday/"
# csv_location = save_location+"temp.folder"
# file_location = save_location+'export_docker.csv'

# df_s3.repartition(1).write.csv(path=csv_location, mode="append", header="true")

# file = dbutils.fs.ls(csv_location)[-1].path
# dbutils.fs.cp(file, file_location)
# dbutils.fs.rm(csv_location, recurse=True)

In [0]:
data = sc.textFile('s3a://sqlsaturday/s3inputfile.dat')

In [0]:
data.count()

In [0]:
data = sc.parallelize(
    [('Amber', 22), ('Alfred', 23), ('Skye',4), ('Albert', 12), 
     ('Amber', 9)])

In [0]:
data_from_file = sc.\
    textFile('./data/VS14MORT.txt.gz', 
        4)

In [0]:
data_from_file.count()

In [0]:
data_from_file.take(10)